<a href="https://colab.research.google.com/github/sharonibejih/leb/blob/training_notebooks/LebForSTTWithMlflow_GitHubVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install jiwer
!pip install evaluate
!pip install accelerate -U
!pip install transformers[torch]
!git clone https://github.com/sunbirdai/leb.git
!pip install -r leb/requirements.txt
!pip install datasets==2.16.1
!pip install mlflow

In [4]:
from torch import nn
import torch
from transformers import (
    AutoFeatureExtractor,
    AutoModelForCTC,
    AutoProcessor,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    is_apex_available,
    set_seed,
)
from dataclasses import dataclass, field
from typing import Union, List, Dict
import string
import datasets
import numpy as np
import leb.dataset
import yaml
import evaluate
import mlflow
from getpass import getpass
import os
from leb.utils import DataCollatorCTCWithPadding as dcwp

# ASR data example

In [5]:
# languages currently available in SALT multispeaker STT dataset
languages = {
    "acholi": "ach",
    "lugbara": "lgg",
    "luganda": "lug",
    "ateso": "teo",
    "runyankole": "nyn",
    "english": "eng"

}

# choose a language of interest
language = languages["acholi"]

# define the directory for output files
project_folder = "./stt"

if not os.path.exists(project_folder):
  %mkdir $project_folder
%cd $project_folder

yaml_config = '''
common_source: &common_source
  type: speech
  language: "{language}"
  preprocessing:
    - set_sample_rate:
        rate: 16_000

common_target: &common_target
  type: text
  language: "{language}"
  preprocessing:
    - lower_case
    - clean_and_remove_punctuation

training_args:
    output_dir: "{project_folder}"
    per_device_train_batch_size: 2
    evaluation_strategy: steps
    num_train_epochs: 5
    max_steps: 8000
    gradient_checkpointing: True
    fp16: True
    save_steps: 1000
    eval_steps: 1000
    logging_steps: 1000
    learning_rate: 0.003
    warmup_steps: 100
    save_total_limit: 2
    # push_to_hub: True
    load_best_model_at_end: True
    metric_for_best_model: wer
    greater_is_better: False
    weight_decay: 0.01

Wav2Vec2ForCTC_args:
    attention_dropout: 0.0
    hidden_dropout: 0.0
    feat_proj_dropout: 0.0
    layerdrop: 0.0
    ctc_loss_reduction: mean
    ignore_mismatched_sizes: True

train:
    huggingface_load:
        - path: Sunbird/salt
          name: multispeaker-{language}
          split: train

    source: *common_source
    target: *common_target
validation:
    huggingface_load:
        - path: Sunbird/salt
          name: multispeaker-{language}
          split: dev

    source: *common_source
    target: *common_target

'''

yaml_config = yaml_config.format(
    project_folder=project_folder,
    language=language
)

config = yaml.safe_load(yaml_config)
train_ds = leb.dataset.create(config['train'])
valid_ds = leb.dataset.create(config['validation'])

/content/drive/MyDrive/stt


In [6]:
## HELPER FUNCTIONS

# Create dict for vocabulary
def extract_all_chars(batch):
    all_text = " ".join(batch["target"])
    vocab = list(set(all_text))
    return {"vocab": vocab, "all_text": [all_text]}

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    batch["input_values"] = processor(
        batch["source"], sampling_rate=16000
    ).input_values
    # Setup the processor for targets
    batch["labels"] = processor(text=batch["target"]).input_ids

    return batch

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [7]:
vocab_dict = {}

for item in train_ds:
    result = extract_all_chars(item)
    for char in result["vocab"]:
        vocab_dict[char] = 1

vocab_list = list(vocab_dict.keys())
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4811 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/96 [00:00<?, ? examples/s]

In [8]:
vocab_dict["|"] = vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [9]:
new_vocab_dict = {language: vocab_dict}

In [10]:
new_vocab_dict

{'ach': {'b': 0,
  't': 1,
  'a': 2,
  'p': 3,
  'i': 4,
  'o': 5,
  'e': 6,
  'm': 7,
  'l': 8,
  'g': 9,
  'r': 10,
  'k': 11,
  ' ': 12,
  'y': 13,
  'n': 14,
  'd': 15,
  'u': 16,
  'w': 17,
  'c': 18,
  'j': 19,
  's': 20,
  'f': 21,
  'z': 22,
  'v': 23,
  'x': 24,
  'h': 25,
  'q': 26,
  '|': 12,
  '[UNK]': 28,
  '[PAD]': 29}}

In [11]:
import json
with open("vocab.json", "w") as vocab_file:
    json.dump(new_vocab_dict, vocab_file)

In [12]:
final_train_dataset = train_ds.map(
    prepare_dataset,
    batch_size=4,
    batched=True,
)

In [13]:
final_val_dataset = valid_ds.map(
    prepare_dataset,
    batch_size=4,
    batched=True,
)

In [14]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", target_lang=language)
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [15]:
wer_metric = evaluate.load('wer')

In [16]:
# data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
data_collator = dcwp(processor=processor, padding=True)

In [17]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/mms-1b-all",
  **config["Wav2Vec2ForCTC_args"],
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

In [18]:
model.gradient_checkpointing_enable()
model.init_adapter_layers()
model.freeze_base_model()

adapter_weights = model._get_adapters()
for param in adapter_weights.values():
    param.requires_grad = True

In [19]:
training_args = TrainingArguments(
  **config["training_args"],
    report_to="none"
)

In [20]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=final_train_dataset,
    eval_dataset=final_val_dataset,
    tokenizer=processor.feature_extractor,
)

__Connect to MLflow Server Using Sunbird Server__

In [22]:
service_account = "mlflow-server@sb-gcp-project-01.iam.gserviceaccount.com"
json_key_name = "sb-gcp-project-01-de9e848cf5a8.json" # ensure file is in working dir

!gcloud auth activate-service-account $service_account --key-file=$json_key_name

# Set the Google Cloud credentials, with storage access
GOOGLE_APPLICATION_CREDENTIALS = json_key_name
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

# Set MLflow tracking credentials
MLFLOW_TRACKING_USERNAME = getpass('Enter the MLFLOW_TRACKING_USERNAME: ') # enter your provided username
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME

MLFLOW_TRACKING_PASSWORD = getpass('Enter the MLFLOW_TRACKING_PASSWORD: ') # enter your provided password
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD

# Set the MLflow tracking URI
mlflow.set_tracking_uri('https://mlflow-sunbird-ce0ecfc14244.herokuapp.com/')

Activated service account credentials for: [mlflow-server@sb-gcp-project-01.iam.gserviceaccount.com]
Enter the JSON path to GCP Service Key: ··········
Enter the MLFLOW_TRACKING_USERNAME: ··········
Enter the MLFLOW_TRACKING_PASSWORD: ··········


__Connect to MLflow Server Locally and Routing to Ngrok (Alternative)__

In [ ]:
# run mlflow tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

# Terminate open tunnels if exist
ngrok.kill()

# Click this URL to get your authtoken https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ') # paste the token you copied
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [23]:
# provide an experiment name
experiment_name = "stt"
try:
    mlflow.create_experiment(experiment_name)
    mlflow.set_experiment(experiment_name)
except Exception:
    mlflow.set_experiment(experiment_name)

In [24]:
with mlflow.start_run(run_name=f"leb-{language}-stt") as run:

    mlflow.set_tag("developer", "your_name")

    train_output = trainer.train()

    # evaluate the model to get the latest metrics including WER
    eval_metrics = trainer.evaluate()

    mlflow.log_params({
"training_args": config["training_args"],
        "Wav2Vec2ForCTC_args": config["Wav2Vec2ForCTC_args"],
        "pretrained_model":"facebook/mms-1b-all"
    })

    # Log metrics after training
    mlflow.log_metrics({
        "train_loss": train_output.training_loss,
        "wer": eval_metrics["eval_wer"]
        # Add more metrics as needed
    })

    # Save and log the model
    trainer.save_model()

    # save model artifacts to mlflow
    artifact_path = "model_artifacts" # you can use any name
    mlflow.log_artifact(f"{experiment_name}/config.json", artifact_path)
    mlflow.log_artifact(f"{experiment_name}/preprocessor_config.json", artifact_path)
    mlflow.log_artifact(f"{experiment_name}/training_args.bin", artifact_path)
    mlflow.log_artifact(f"{experiment_name}/model.safetensors", artifact_path)


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
1000,1.407400,0.529559,0.507082
2000,0.678100,0.405224,0.407932
3000,0.434800,0.374388,0.454202
4000,0.569700,0.359813,0.406988
5000,0.388200,0.351032,0.408876
6000,0.309800,0.323310,0.355052
7000,0.522500,0.319898,0.372049
8000,0.339900,0.307022,0.395656


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.